Импорт библиотек и объявление путей к изображению и водяному знаку

In [1]:
from PIL import Image
import numpy as np
import hashlib
import time

WATERMARTK_PATH = 'pixel_knight.jpg'
# WATERMARTK_PATH = 'pizza.jpg'
IMAGE_PATH = 'lena_std.tif'
# IMAGE_PATH = 'Airplane.tiff'
# IMAGE_PATH = 'House.tiff'
# IMAGE_PATH = 'Peppers.tiff'
# IMAGE_PATH = 'Mandrill.tiff'
# IMAGE_PATH = 'Sailboat.tiff'
# IMAGE_PATH = 'Splash.tiff'


Реализуем функции, необходимые для обработки и встраивания водяного знака
1) Функция генерации матрицы Адамара, по заданной длине ребра

In [2]:
def hadamar_matrix(n):
    """
    Функция генерации матрицы Адамара размерности n.
    Возвращает матрицу типа np.array.
    """
    # Создание матрицы с единицами в левом верхнем углу
    hadamard = np.array([1])
    # Увеличение матрицы до нужной размерности n
    while hadamard.shape[0] < n:
        # Расширение матрицы из левого верхнего угла
        hadamard = np.vstack([np.hstack([hadamard, hadamard]), np.hstack([hadamard, -hadamard])])
    return hadamard


2) Функция прямого и обратного преобразования Уолша-Адамара для двумерных матриц

In [3]:
# функция, выполняющая прямое преобразование Уолша-Адамара
def WHT(matrix):
    hadamar = hadamar_matrix(4)
    return 0.25 * np.matmul(hadamar, matrix)


# функция, выполняющая обратное пеобразование Уолша-Адамара
def inverse_WHT(matrix):
    hadamar = hadamar_matrix(4)
    return np.matmul(hadamar, matrix)

3) Функция прямой и обратной трансформации Арнольда, для матрицы пикселей водяного знака, принимающие на вход координаты пикселя, матрицу Арнольда, и длину ребра изображения водяного знака. Возвращают новое значение координат для пикселя.

In [4]:
def arnold_transform(x, y, matr, n):
    xy = np.array([x, y])
    a = np.array(matr)
    xy_transformed = np.matmul(xy, a) % n
    return int(xy_transformed[0]), int(xy_transformed[1])


def invers_arnold_transform(x_transformed, y_transformed, matr, n):
    inversed_matr = np.linalg.matrix_power(matr, -1)
    xy_transformed = np.array([x_transformed, y_transformed])
    xy = np.matmul(xy_transformed, inversed_matr) % n
    return int(xy[0]), int(xy[1])


4) Алгоритм MD5 для изменения порядка блоков изображения, в которые будет встроен водяной знак

In [5]:
def generate_ji(i, X, Y, k1, k2):
    v = i // X
    u = i % X
    ji = v * X + u
    for _ in range(10):
        v = (v + int(hashlib.md5(str(u + k1).encode()).hexdigest(), 16)) % Y
        u = (u + int(hashlib.md5(str(v + k2).encode()).hexdigest(), 16)) % X
        ji = v * X + u
    return ji


# Example usage
block_size = 4
X = Y = 512 // block_size
total_carrier_units = X * Y
indexes_of_blocks_for_watermark_embedding = [generate_ji(i, X, Y, 123, 456) for i in range(total_carrier_units)]
# print(indexes_of_blocks_for_watermark_embedding)
# for i, n in enumerate(indexes_of_blocks_for_watermark_embedding):
#     indexes_of_blocks_for_watermark_embedding[i] = [n//128, n % 128]
# print(len(indexes_of_blocks_for_watermark_embedding) == len((set(indexes_of_blocks_for_watermark_embedding))))
# iss = [generate_ji(i, X, Y, 123, 456 for i in range]

Подготовка водяного знака к встраиванию

In [6]:
R = []
G = []
B = []
watermark = Image.open(WATERMARTK_PATH)
pixels = watermark.load()
for i in range(watermark.size[0]):
    r = []
    g = []
    b = []
    for j in range(watermark.size[1]):
        r.append(pixels[i, j][0])
        g.append(pixels[i, j][1])
        b.append(pixels[i, j][2])
    R.append(r)
    G.append(g)
    B.append(b)


# Трансформация (скремблирование) Арнольда
R_scrambled = R.copy()
G_scrambled = G.copy()
B_scrambled = B.copy()

# for i in range(len(R)):
#     for j in range(len(R[0])):
#         x, y = arnold_transform(i, j, [[1, 1], [1, 2]], 32)
#         R_scrambled[i][j] = R[x][y]
#         G_scrambled[i][j] = G[x][y]
#         B_scrambled[i][j] = B[x][y]

# Преобразование сетки пикселей в двоичную последовательность
R_bin = ''
G_bin = ''
B_bin = ''
for i in range(len(R)):
    for j in range(len(R[0])):
        R_bin += '0' * (8 - len(bin(R_scrambled[i][j])[2:])) + bin(R_scrambled[i][j])[2:]
        G_bin += '0' * (8 - len(bin(G_scrambled[i][j])[2:])) + bin(G_scrambled[i][j])[2:]
        B_bin += '0' * (8 - len(bin(B_scrambled[i][j])[2:])) + bin(B_scrambled[i][j])[2:]
#УДВЛИТЬ ПОСЛЕ ОТЛАДКИ
# R_bin = '01'
# print(R_bin)

Подготовка изображения к встраиванию

In [7]:
host_image = Image.open(IMAGE_PATH)
host_pixels = host_image.load()
# x, y = host_image.size
R = []
G = []
B = []

# разбиние сетки пикселей изображения на блоки 4х4 пикселя
for i in range(0, host_image.size[0], 4):
    # Проход по столбцам с шагом 4
    for j in range(0, host_image.size[1], 4):
        # Создание блока
        rblock = []
        gblock = []
        bblock = []
        # Проход по строкам блока
        for k in range(i, i + 4):
            rline = []
            gline = []
            bline = []
            # Проход по столбцам блока
            for l in range(j, j + 4):
                # Добавление элемента в блок
                rline.append(host_pixels[k, l][0])
                gline.append(host_pixels[k, l][1])
                bline.append(host_pixels[k, l][2])
            rblock.append(rline)
            gblock.append(gline)
            bblock.append(bline)
        # Добавление блока в список
        R.append(list(rblock.copy()))
        G.append(list(gblock.copy()))
        B.append(list(bblock.copy()))
print(len(R))
# определение порядка пикселей, путем применения алгоритма MD5

# Step 4 obtaining the frequency domain coefcient through  WHT
for i in range(len(R)):
    R[i] = WHT(R[i])
    G[i] = WHT(G[i])
    B[i] = WHT(B[i])
print(R)
# print(G)
# print(B)

16384
[array([[224.5, 224.5, 224.5, 224.5],
       [  0. ,   0. ,   0. ,   0. ],
       [  1.5,   1.5,   1.5,   1.5],
       [  0. ,   0. ,   0. ,   0. ]]), array([[224.5 , 225.75, 226.25, 223.25],
       [  0.  ,   1.25,   0.25,   1.25],
       [  1.5 ,   1.25,   1.75,  -0.25],
       [  0.  ,  -1.25,  -0.25,  -1.25]]), array([[224.5, 223. , 223. , 224. ],
       [  0. ,   0.5,   0. ,  -0.5],
       [  0.5,   0. ,   1. ,  -1. ],
       [  0. ,  -0.5,   0. ,   0.5]]), array([[226.5 , 226.75, 226.25, 227.25],
       [ -0.5 ,  -0.25,   0.75,   0.25],
       [  0.5 ,   1.25,   1.75,   0.75],
       [  0.5 ,   0.25,  -0.75,  -0.25]]), array([[226.25, 225.25, 225.75, 224.25],
       [ -0.25,   0.25,  -0.25,   0.25],
       [  1.75,   0.75,   0.25,   1.75],
       [  0.25,  -0.25,   0.25,  -0.25]]), array([[224.  , 225.75, 225.75, 225.25],
       [ -0.5 ,   0.75,  -0.25,   0.75],
       [  0.  ,   1.25,   1.25,   0.75],
       [  0.5 ,  -0.75,   0.25,  -0.75]]), array([[225.  , 224.25, 224.2

Встраивание битовой последовательности в изображение

In [8]:
start = time.time()
D = 5  # Error parameter
T = 5  # Quantization step size
ind = 0




for i in range(0, len(R_bin), 2):



    r_block_pow_line = R[indexes_of_blocks_for_watermark_embedding[ind]][0].copy()
    g_block_pow_line = G[indexes_of_blocks_for_watermark_embedding[ind]][0].copy()
    b_block_pow_line = B[indexes_of_blocks_for_watermark_embedding[ind]][0].copy()

    r_power_line = r_block_pow_line.copy()
    g_power_line = g_block_pow_line.copy()
    b_power_line = b_block_pow_line.copy()

    w_r1 = int(R_bin[i])
    w_r2 = int(R_bin[i + 1])
    w_g1 = int(G_bin[i])
    w_g2 = int(G_bin[i + 1])
    w_b1 = int(B_bin[i])
    w_b2 = int(B_bin[i + 1])

    avg_r12 = (r_block_pow_line[0] + r_block_pow_line[1]) / 2
    avg_r34 = (r_block_pow_line[2] + r_block_pow_line[3]) / 2
    avg_g12 = (g_block_pow_line[0] + g_block_pow_line[1]) / 2
    avg_g34 = (g_block_pow_line[2] + g_block_pow_line[3]) / 2
    avg_b12 = (b_block_pow_line[0] + b_block_pow_line[1]) / 2
    avg_b34 = (b_block_pow_line[2] + b_block_pow_line[3]) / 2

    # Для красного канала
    if w_r1 == 1 and abs(r_power_line[0]) - abs(r_power_line[1]) <= D:
        r_block_pow_line[0] = np.sign(r_power_line[0]) * (avg_r12 + T / 2)
    elif w_r1 == 0 and abs(r_power_line[1]) - abs(r_power_line[0]) <= D:
        r_block_pow_line[0] = np.sign(r_power_line[0]) * (avg_r12 - T / 2)

    if w_r1 == 0 and abs(r_power_line[1]) - abs(r_power_line[0]) <= D:
        r_block_pow_line[1] = np.sign(r_power_line[1]) * (avg_r12 + T / 2)
    elif w_r1 == 1 and abs(r_power_line[0]) - abs(r_power_line[1]) <= D:
        r_block_pow_line[1] = np.sign(r_power_line[1]) * (avg_r12 - T / 2)

    if w_r2 == 1 and abs(r_power_line[2]) - abs(r_power_line[3]) <= D:
        r_block_pow_line[2] = np.sign(r_power_line[2]) * (avg_r34 + T / 2)
    elif w_r2 == 0 and abs(r_power_line[3]) - abs(r_power_line[2]) <= D:
        r_block_pow_line[2] = np.sign(r_power_line[2]) * (avg_r34 - T / 2)

    if w_r2 == 0 and abs(r_power_line[3]) - abs(r_power_line[2]) <= D:
        r_block_pow_line[3] = np.sign(r_power_line[3]) * (avg_r34 + T / 2)
    elif w_r2 == 1 and abs(r_power_line[2]) - abs(r_power_line[3]) <= D:
        r_block_pow_line[3] = np.sign(r_power_line[3]) * (avg_r34 - T / 2)

    # Для зеленого канала
    if w_g1 == 1 and abs(g_power_line[0]) - abs(g_power_line[1]) <= D:
        g_block_pow_line[0] = np.sign(g_power_line[0]) * (avg_g12 + T / 2)
    elif w_g1 == 0 and abs(g_power_line[1]) - abs(g_power_line[0]) <= D:
        g_block_pow_line[0] = np.sign(g_power_line[0]) * (avg_g12 - T / 2)

    if w_g1 == 0 and abs(g_power_line[1]) - abs(g_power_line[0]) <= D:
        g_block_pow_line[1] = np.sign(g_power_line[1]) * (avg_g12 + T / 2)
    elif w_g1 == 1 and abs(g_power_line[0]) - abs(g_power_line[1]) <= D:
        g_block_pow_line[1] = np.sign(g_power_line[1]) * (avg_g12 - T / 2)

    if w_g2 == 1 and abs(g_power_line[2]) - abs(g_power_line[3]) <= D:
        g_block_pow_line[2] = np.sign(g_power_line[2]) * (avg_g34 + T / 2)
    elif w_g2 == 0 and abs(g_power_line[3]) - abs(g_power_line[2]) <= D:
        g_block_pow_line[2] = np.sign(g_power_line[2]) * (avg_g34 - T / 2)

    if w_g2 == 0 and abs(g_power_line[3]) - abs(g_power_line[2]) <= D:
        g_block_pow_line[3] = np.sign(g_power_line[3]) * (avg_g34 + T / 2)
    elif w_g2 == 1 and abs(g_power_line[2]) - abs(g_power_line[3]) <= D:
        g_block_pow_line[3] = np.sign(g_power_line[3]) * (avg_g34 - T / 2)

    # Для голубого канала
    if w_b1 == 1 and abs(b_power_line[0]) - abs(b_power_line[1]) <= D:
        b_block_pow_line[0] = np.sign(b_power_line[0]) * (avg_b12 + T / 2)
    elif w_b1 == 0 and abs(b_power_line[1]) - abs(b_power_line[0]) <= D:
        b_block_pow_line[0] = np.sign(b_power_line[0]) * (avg_b12 - T / 2)

    if w_b1 == 0 and abs(b_power_line[1]) - abs(b_power_line[0]) <= D:
        b_block_pow_line[1] = np.sign(b_power_line[1]) * (avg_b12 + T / 2)
    elif w_b1 == 1 and abs(b_power_line[0]) - abs(b_power_line[1]) <= D:
        b_block_pow_line[1] = np.sign(b_power_line[1]) * (avg_b12 - T / 2)

    if w_b2 == 1 and abs(b_power_line[2]) - abs(b_power_line[3]) <= D:
        b_block_pow_line[2] = np.sign(b_power_line[2]) * (avg_b34 + T / 2)
    elif w_b2 == 0 and abs(b_power_line[3]) - abs(b_power_line[2]) <= D:
        b_block_pow_line[2] = np.sign(b_power_line[2]) * (avg_b34 - T / 2)

    if w_b2 == 0 and abs(b_power_line[3]) - abs(b_power_line[2]) <= D:
        b_block_pow_line[3] = np.sign(b_power_line[3]) * (avg_b34 + T / 2)
    elif w_b2 == 1 and abs(b_power_line[2]) - abs(b_power_line[3]) <= D:
        b_block_pow_line[3] = np.sign(b_power_line[3]) * (avg_b34 - T / 2)




    R[indexes_of_blocks_for_watermark_embedding[ind]][0] = r_block_pow_line.copy()
    G[indexes_of_blocks_for_watermark_embedding[ind]][0] = g_block_pow_line.copy()
    B[indexes_of_blocks_for_watermark_embedding[ind]][0] = b_block_pow_line.copy()
    ind += 1

for i in range(len(R)):
    R[i] = inverse_WHT(R[i])
    G[i] = inverse_WHT(G[i])
    B[i] = inverse_WHT(B[i])
print(R)

end = time.time() - start
# Разбиваем блоки обратно в двумерную матрицу пикселей
R_pix = []
G_pix = []
B_pix = []
a = []
b = []
c = []
d = []
for i in range(len(R)):
    a.extend(R[i][0])
    b.extend(R[i][1])
    c.extend(R[i][2])
    d.extend(R[i][3])
    if len(a) == 512:
        R_pix.append(a)
        R_pix.append(b)
        R_pix.append(c)
        R_pix.append(d)
        a = []
        b = []
        c = []
        d = []



a = []
b = []
c = []
d = []
for i in range(len(G)):
    a.extend(G[i][0])
    b.extend(G[i][1])
    c.extend(G[i][2])
    d.extend(G[i][3])
    if len(a) == 512:
        G_pix.append(a)
        G_pix.append(b)
        G_pix.append(c)
        G_pix.append(d)
        a = []
        b = []
        c = []
        d = []

a = []
b = []
c = []
d = []
for i in range(len(B)):
    a.extend(B[i][0])
    b.extend(B[i][1])
    c.extend(B[i][2])
    d.extend(B[i][3])
    if len(a) == 512:
        B_pix.append(a)
        B_pix.append(b)
        B_pix.append(c)
        B_pix.append(d)
        a = []
        b = []
        c = []
        d = []
print(R_pix)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Сохранение изображения с встроенным водяным знаком

In [ ]:
host_image_with_whatermark = Image.new("RGB", (512, 512))
host_with_whater_pix = host_image_with_whatermark.load()

for i in range(512):
    for j in range(512):
        host_with_whater_pix[i, j] = (round(R_pix[i][j]), round(G_pix[i][j]), round(B_pix[i][j]))
        print(host_with_whater_pix[i, j], end=' ')
host_image_with_whatermark.save('host_image_with_whatermark.j2k')


Извлечение водяного знака из изображения

In [ ]:

host_with_whater_pix = host_image_with_whatermark.load()

R = []
G = []
B = []
# Разбиение изображения на блоки 4 на 4 пикселя
for i in range(0, 512, 4):
    # Проход по столбцам с шагом 4
    for j in range(0, 512, 4):
        # Создание блока
        rblock = []
        gblock = []
        bblock = []
        # Проход по строкам блока
        for k in range(i, i + 4):
            rline = []
            gline = []
            bline = []
            # Проход по столбцам блока
            for l in range(j, j + 4):
                # Добавление элемента в блок
                rline.append(host_with_whater_pix[k, l][0])
                gline.append(host_with_whater_pix[k, l][1])
                bline.append(host_with_whater_pix[k, l][2])
            rblock.append(rline)
            gblock.append(gline)
            bblock.append(bline)
        # Добавление блока в список
        R.append(list(rblock.copy()))
        G.append(list(gblock.copy()))
        B.append(list(bblock.copy()))


print(R)
# Преобразование полученных блоков методом Уоша-Адамара
for i in range(len(R)):
    R[i] = WHT(R[i])
    G[i] = WHT(G[i])
    B[i] = WHT(B[i])

# Извлечение бинарной последовательности из блоков изображения
R_bin = []
G_bin = []
B_bin = []
r_bin = ''
g_bin = ''
b_bin = ''
ind = 0
for i in range(4 * 32 ** 2):
    r_block_pow_line = R[indexes_of_blocks_for_watermark_embedding[ind]][0]
    g_block_pow_line = G[indexes_of_blocks_for_watermark_embedding[ind]][0]
    b_block_pow_line = B[indexes_of_blocks_for_watermark_embedding[ind]][0]
    ind += 1
    # Извлечение информации из красного канала
    if r_block_pow_line[0] <= r_block_pow_line[1]:
        w_r1 = '0'
    else:
        w_r1 = '1'
    if r_block_pow_line[2] <= r_block_pow_line[3]:
        w_r2 = '0'
    else:
        w_r2 = '1'
    # Извлечение информации из зеленого канала
    if g_block_pow_line[0] <= g_block_pow_line[1]:
        w_g1 = '0'
    else:
        w_g1 = '1'
    if g_block_pow_line[2] <= g_block_pow_line[3]:
        w_g2 = '0'
    else:
        w_g2 = '1'
    # Извлечение информации из синего канала
    if b_block_pow_line[0] <= b_block_pow_line[1]:
        w_b1 = '0'
    else:
        w_b1 = '1'
    if b_block_pow_line[2] <= b_block_pow_line[3]:
        w_b2 = '0'
    else:
        w_b2 = '1'
    r_bin = r_bin + w_r1 + w_r2
    g_bin = g_bin + w_g1 + w_g2
    b_bin = b_bin + w_b1 + w_b2
    if len(r_bin) == 8:
        R_bin.append(int(r_bin, 2))
        G_bin.append(int(g_bin, 2))
        B_bin.append(int(b_bin, 2))
        r_bin = ''
        g_bin = ''
        b_bin = ''


print(w_r1)
print(w_r2)
# Выполняем обратную трансформацию Арнольда
R = []
G = []
B = []
r_help = []
g_help = []
b_help = []
for i in range(len(R_bin)):
    r_help.append(R_bin[i])
    g_help.append(G_bin[i])
    b_help.append(B_bin[i])
    if len(r_help) == 32:
        R.append(r_help.copy())
        G.append(g_help.copy())
        B.append(b_help.copy())
        r_help = []
        g_help = []
        b_help = []

R_scrambled = R[:]
G_scrambled = G[:]
B_scrambled = B[:]

for i in range(len(R)):
    for j in range(len(R[0])):
        x, y = invers_arnold_transform(i, j, [[1, 1], [1, 2]], 32)
        R_scrambled[i][j] = R[x][y]
        G_scrambled[i][j] = G[x][y]
        B_scrambled[i][j] = B[x][y]

embedded_whatermark_img = Image.new("RGB", (32, 32))
embedded_whtr_img_pix = embedded_whatermark_img.load()
for i in range(32):
    for j in range(32):
        embedded_whtr_img_pix[i, j] = (R_scrambled[i][j], G_scrambled[i][j], B_scrambled[i][j])
embedded_whatermark_img.save('embedded_whatermark.tif')


In [ ]:
R = []
G = []
B = []
h1 = []
h2 = []
h3 = []
for i in range(len(R_bin)):
    h1.append(R_bin[i])
    h2.append(G_bin[i])
    h3.append(B_bin[i])
    if len(h1) == 8:
        R.append(h1.copy())
        G.append(h2.copy())
        B.append(h3.copy())
        h1 = []
        h2 = []
        h3 = []

h1 = []
h2 = []
h3 = []
R_pix = []
G_pix = []
B_pix = []

for i in range(len(R)):
    h1.extend(R[i])
    h2.extend(G[i])
    h3.extend(B[i])
    if len(h1) == 32:
        R_pix.append(h1.copy())
        G_pix.append(h2.copy())
        B_pix.append(h3.copy())
        h1 = []
        h2 = []
        h3 = []

print(R_pix)
embedded_whatermark_img = Image.new("RGB", (32, 32))
embedded_whtr_img_pix = embedded_whatermark_img.load()
for i in range(32):
    for j in range(32):
        embedded_whtr_img_pix[i, j] = (R_pix[i][j], G_pix[i][j], B_pix[i][j])
embedded_whatermark_img.save('embedded_whatermark.tif')

In [9]:
print(end)

3.0710091590881348
